# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [4]:
import napari
import os, glob
from macrohet import dataio #, visualise, notify
import numpy as np
import os
import re
import numpy as np
import cv2
import btrack
import zarr


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [5]:
%%time
expt_ID = 'ND0005'

base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 3.47 s, sys: 36.8 ms, total: 3.51 s
Wall time: 3.5 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135257497,0,2024-10-06T18:14:34.337+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135257497,0,2024-10-06T18:14:34.57+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135259494,0,2024-10-06T18:14:34.913+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135259494,0,2024-10-06T18:14:35.133+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135261506,0,2024-10-06T18:14:35.477+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31093,0404K192F9P1R2,Ok,r04c04f09p01-ch2sk192fk1fl1.tiff,4,4,9,1,191,2,1,...,0,0.135189295,57306.39,2024-10-07T10:10:41.33+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31094,0404K192F9P2R1,Ok,r04c04f09p02-ch1sk192fk1fl1.tiff,4,4,9,2,191,1,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.677+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31095,0404K192F9P2R2,Ok,r04c04f09p02-ch2sk192fk1fl1.tiff,4,4,9,2,191,2,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.893+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31096,0404K192F9P3R1,Ok,r04c04f09p03-ch1sk192fk1fl1.tiff,4,4,9,3,191,1,1,...,4E-06,0.135193303,57306.39,2024-10-07T10:10:42.237+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

### Load using Zarr

In [6]:
acq_ID = (3, 4)

In [7]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

zarr_group = zarr.open(image_dir, mode='r')

In [6]:
%%time
images = zarr_group.images[:,1,1,...]
images.shape

CPU times: user 9.75 s, sys: 4.12 s, total: 13.9 s
Wall time: 3.67 s


(150, 6048, 6048)

In [9]:
zarr_group.images

<zarr.core.Array '/images' (192, 2, 3, 6048, 6048) uint16 read-only>

In [ ]:
%%time
images_max_proj = np.max(zarr_group.images, axis = 2)

In [ ]:
images_max_proj.shape

In [7]:
images.shape

(150, 6048, 6048)

#### Load downscaled version from mp4 videos

In [8]:
mp4_path = glob.glob(f'/mnt/SYNO/videos/macrohet_videos/{expt_ID}/{expt_ID}_{acq_ID}*')[0]

# Launch napari

In [ ]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, max_proj')


viewer.add_image(images_max_proj,
                 channel_axis = 1, 
                 colormap=['magenta', 'green'],
                 blending = 'additive', 
                 contrast_limits=[[350, 1000],[0, 7000]]
                )

In [14]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/10/21 12:25:01 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0005/labels/cpv3/(3, 3).h5...
[INFO][2024/10/21 12:25:28 pm] Loading segmentation (192, 6048, 6048)
[INFO][2024/10/21 12:25:28 pm] Loading tracks/obj_type_1
[INFO][2024/10/21 12:25:28 pm] Loading LBEP/obj_type_1
[INFO][2024/10/21 12:25:29 pm] Loading objects/obj_type_1 (50810, 5) (50810 filtered: None)
[INFO][2024/10/21 12:25:30 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0005/labels/cpv3/(3, 3).h5


### Add tracks

In [10]:
filtered_tracks = [track for track in tracks if len(track) > 35]

In [27]:
recoloured_seg = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

In [15]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [16]:
viewer.add_tracks(napari_tracks, scale=(1, 5.04, 5.04), name = 'unfiltered tracks')

<Tracks layer 'unfiltered tracks' at 0x7fc9b2ead570>

### Add segmentation

In [17]:
viewer.add_labels(segmentation)
# viewer.add_labels(recoloured_seg)

<Labels layer 'segmentation' at 0x7fc9b2da5270>

In [28]:
viewer.add_image(images_max_proj[-5],
                 channel_axis = 0, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400],[0, 1000]]
                )

[<Image layer 'Image [4]' at 0x7fa481516460>,
 <Image layer 'Image [5]' at 0x7fa4823087f0>]

In [ ]:
viewer.camera.reset()
`

In [14]:
visualise.highlight_cell(126, viewer, filtered_tracks, scale_factor=5.02)

<Points layer 'cell 126' at 0x7f248be4fa60>

In [20]:
for layer in viewer.layers:
    viewer.layers[layer.name].scale = (10, 1, 1)

In [21]:
viewer.layers[2].name

'filtered'

In [22]:
viewer.layers[2].scale = (10,5.04, 5.04)

# Saving out as a list of JPG files for SAM2 testing

In [19]:
images_max_proj.shape

(192, 2, 6048, 6048)

In [24]:
from tqdm.auto import tqdm
from PIL import Image
import os
import cv2
import numpy as np

# Assuming `images_max_proj` is a numpy array of images with shape (n_images, channels, height, width)
output_dir = f"/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/jpegs/{acq_ID}"  # Directory to save the images
os.makedirs(output_dir, exist_ok=True)

# Ensure the MP4 output directory exists
mp4_dir = f"/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/mp4/{acq_ID}"
os.makedirs(mp4_dir, exist_ok=True)

# Set the MP4 output path
output_video_path = os.path.join(mp4_dir, f"{acq_ID}.mp4")

# Initialize video writer for the MP4 file using OpenCV
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
fps = 10  # Set frames per second
downscaled_size = (1920, 1920)  # New size for downscaling to 1000x1000

video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, downscaled_size)

# Contrast limits for both channels
contrast_limits = [[350, 1000], [0, 7000]]

# Function to apply contrast stretching
def apply_contrast_limits(image, vmin, vmax):
    image = np.clip(image, vmin, vmax)
    image = (image - vmin) / (vmax - vmin) * 255
    return image.astype(np.uint8)

# Iterate over each image in both channels
for idx, (img_ch1, img_ch2) in enumerate(tqdm(zip(images_max_proj[:, 1, ...], images_max_proj[:, 0, ...]), total = 192, desc="Saving images and creating video")):
    # Apply contrast limits to both channels
    img_ch1_adjusted = apply_contrast_limits(img_ch1, *contrast_limits[1])
    img_ch2_adjusted = apply_contrast_limits(img_ch2, *contrast_limits[0])
    
    # Create an RGB image where channel 1 is green and channel 0 is magenta
    img_rgb = np.zeros((img_ch1.shape[0], img_ch1.shape[1], 3), dtype=np.uint8)
    img_rgb[..., 1] = img_ch1_adjusted  # Green channel for ch1
    img_rgb[..., 0] = img_ch2_adjusted  # Red channel for ch0 (magenta = red + blue)
    img_rgb[..., 2] = img_ch2_adjusted  # Blue channel for ch0

    # Downscale the image to 1000x1000 px
    img_downscaled = cv2.resize(img_rgb, downscaled_size, interpolation=cv2.INTER_AREA)
    
    # Save each downscaled image as a 5-digit frame number in jpg format
    frame_filename = os.path.join(output_dir, f"{idx:05d}.jpg")
    cv2.imwrite(frame_filename, img_downscaled)

    # Write the current downscaled frame to the video
    video_writer.write(cv2.imread(frame_filename))

# Release the video writer to finalize the video file
video_writer.release()

Saving images and creating video:   0%|          | 0/192 [00:00<?, ?it/s]

In [17]:
napari.Viewer().add_image(img)

<Image layer 'img' at 0x7fa33291c5e0>